In [1]:
#nbi:hide_in
from __future__ import print_function
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_moons, make_circles, make_blobs
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoLars,lars_path,LassoLarsCV
from sklearn import datasets
import ipywidgets as widgets
from ipywidgets import IntSlider
from ipywidgets.embed import embed_data
import nbinteract as nbi
from ipywidgets import interact, interactive


In [2]:
#nbi:hide_in
def LarsLasso(Distribution,noise,n_samples,LogAlpha,checkbox1):
#     Noise = Noise/100
    n_samples = int(n_samples)
    centers = 5
    Alpha = np.exp(LogAlpha)
    seed = 3
    np.random.seed(seed)
    if Distribution == "Linear":
        X_mn = np.linspace(0, 20, n_samples)
        m = np.random.random_sample() * 25
        c = np.random.random_sample() * 25
        y_mn = m*X_mn + c + np.random.rand(*X_mn.shape) * noise
        
    if Distribution == "Exponential":
        X_mn = np.linspace(0, 4, n_samples)
        m = np.random.random_sample() * 25
        c = np.random.random_sample() * 25
        y_mn = m*np.exp(X_mn) + c + np.random.rand(*X_mn.shape) * noise
        
    if Distribution == "Logrithmic":
        X_mn = np.linspace(0.5, 25, n_samples)
        m = np.random.random_sample() * 25
        c = np.random.random_sample() * 25
        y_mn = m*np.log(X_mn) + c + np.random.rand(*X_mn.shape) * noise
        
    if Distribution == "Blobs":
        for i in range(centers):
            X_mn = np.random.rand(n_samples) * np.exp(noise)
            y_mn = np.random.rand(n_samples) * np.exp(noise)
            m = np.random.randint(low = -100, high = 100) * 25
            c = np.random.randint(low = -100, high = 100) * 25
            X_mn += m
            y_mn += c
            if i == 0:
                con_X_mn = X_mn.copy()
                con_y_mn = y_mn.copy()
                
            else:
                con_X_mn = np.concatenate((con_X_mn, X_mn))
                con_y_mn = np.concatenate((con_y_mn, y_mn))
                
        
    X = X_mn
    Y = y_mn
    
    X = X/np.max(X)
    O = X
    O_ = np.hstack((np.ones((X.shape[0],1)),np.c_[X]))
    if(checkbox1==True):
        r = LassoLarsCV(cv=5).fit(O_, Y)
        Alpha = r.alpha_
        print('Optimal Alpha = ' + str(Alpha))
    reg = LassoLars(alpha=Alpha)
    reg.fit(O_,Y)
    theta = reg.coef_
    plt.figure(figsize=(20,10))
    plt.scatter(X,Y, s=10, label ='Datapoints')
    y_pred = reg.predict(O_)
    plt.plot(X, y_pred, color='r', label="fitted curve")
    plt.xlabel("X")
    plt.ylabel("y")
#     plt.label()
    plt.title("Lars Lasso Regression with alpha: "+ str(Alpha))
    plt.show()
    #return X,Y, y_pred

# Lars Lasso 

Feature selection is often used to come up with better models with lower computations and better accuracy. Feature selection methods like Forward Selection and Backward Selection look into each of the features and try to add or delete features that are considered essential and less critical respectively. It comes in handy when dealing with high-dimensional data as it takes into consideration the correlation of features with the residual to select the active features. The ```LARS``` algorithm is similar to forward stepwise regression, but instead of including variables at each step, the estimated parameters are increased in a direction equiangular to each one's correlations with the residual. This process is iterated until we run out of all the features.


In [3]:
#nbi:hide_in
# https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_lars.html
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn import datasets
def lars_path(iterations):
    diabetes = datasets.load_diabetes()
    X = diabetes.data
    y = diabetes.target

    _, _, coefs = linear_model.lars_path(X, y, method='lar', verbose=True,max_iter=iterations)

    xx = np.sum(np.abs(coefs.T), axis=1)
    xx /= xx[-1]

    plt.figure(figsize=(20,10))
    plt.plot(xx, coefs.T)
    ymin, ymax = plt.ylim()
    plt.vlines(xx, ymin, ymax, linestyle='dashed')
    plt.xlabel('|coef| / max|coef|')
    plt.ylabel('Coefficients')
    plt.title('LARS Path')
    plt.axis('tight')
    plt.show()
    
widget = interactive(lars_path,
                     iterations = (1, 20, 1))
widget

interactive(children=(IntSlider(value=10, description='iterations', max=20, min=1), Output()), _dom_classes=('…


```LARS LASSO (least absolute shrinkage and selection operator)``` is a regression method that performs both variable selection and regularization to reduce the model complexity for better generalizability. The ``LASSO`` modification to the ``LARS`` algorithm eliminates a particular feature from the active set when the corresponding coefficient nears zero. Thus ``LARS LASSO`` is a combination of forward and backward feature selection methods.


In [4]:
#nbi:hide_in
checkbox1 = widgets.Checkbox(description='Use Optimal Alpha')
checkbox2 = widgets.Checkbox(description='Use Non-linear Data')
widget = interactive(LarsLasso,
                     Distribution=['Linear', 'Exponential','Logrithmic'],#,'Blobs'],
                     noise = (0, 1e2, 1),
                     n_samples = (10, 1e2, 10),
                     LogAlpha = (-10, 10, 0.05),
#                      seed = (0, 10, 1),
                     checkbox1=checkbox1)
widget

interactive(children=(Dropdown(description='Distribution', options=('Linear', 'Exponential', 'Logrithmic'), va…

#### Abhvya Chandra | Apoorv Agnihotri | Jatin Dholakia | Sagar Gupta